<a href="https://colab.research.google.com/github/m-t-abbasi/SpatialPlanning/blob/main/section5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import geopandas as gpd
import numpy as np
import random

# مسیر فایل شیپ را تعیین می کنیم
shapefile_path = "/content/landuse_shp.shp"

# خواندن فایل شیپ
gdf = gpd.read_file(shapefile_path)

# تعداد کل پارسل‌های منطقه
total_parcels = len(gdf)

# شمارش تعداد پارسل‌ها به تفکیک کاربری
landuse_counts = gdf['landuse2'].value_counts().to_dict()

# محاسبه مساحت هر کاربری بر حسب کل مساحت پارسل‌های منطقه
landuse_areas = gdf.groupby('landuse2')['SHAPE_Area'].sum().to_dict()

# تعریف اندازه‌های مختلف بافر
buffer_sizes = [100, 200, 300]

# دیکشنری برای ذخیره میانگین نسبت‌ها بر اساس کاربری و بافر
mean_ratios = {}

# تعداد پارسل‌هایی که برای محاسبه استفاده می‌شود
sample_size = 10

# پیمایش تک تک کاربری‌ها
for landuse_type in landuse_counts.keys():
    mean_ratios[landuse_type] = {}  # دیکشنری برای ذخیره میانگین نسبت‌ها بر اساس بافر
    
    # پیمایش تک تک اندازه‌های بافر
    for buffer_size in buffer_sizes:
        ratios = []  # لیستی برای ذخیره نسبت‌های محاسبه شده
        
        # انتخاب تصادفی ۱۰ پارسل برای محاسبه
        random_parcel_indices = random.sample(range(total_parcels), sample_size)
        
        # پیمایش تک تک پارسل‌های انتخاب شده
        for index in random_parcel_indices:
            row = gdf.iloc[index]
            parcel_id = row['OBJECTID']  # شماره پارسل
            
            buffer = row.geometry.buffer(buffer_size)
            
            # پیدا کردن پارسل‌هایی که کاملاً در داخل هر بافر قرار دارند
            parcels_within_buffer = gdf[gdf.geometry.within(buffer)]
            
            # شمارش تعداد پارسل‌ها به تفکیک کاربری
            landuse_counts_within_buffer = parcels_within_buffer['landuse2'].value_counts().to_dict()
            
            # محاسبه مساحت هر کاربری در هر بافر
            landuse_areas_within_buffer = parcels_within_buffer.groupby('landuse2')['SHAPE_Area'].sum().to_dict()
            
            # محاسبه نسبت آماری برای هر کاربری
            S_di = sum(landuse_areas_within_buffer.values())
            N_k = landuse_counts[landuse_type]
            S_k = landuse_areas[landuse_type]
            N = len(gdf)
            S = sum(landuse_areas.values())
            
            ratio = ((landuse_counts_within_buffer.get(landuse_type, 0) * landuse_areas_within_buffer.get(landuse_type, 0)) / (N_k * S_di)) / ((N_k * S_k) / (N * S))
            ratios.append(ratio)
    
        # محاسبه میانگین نسبت‌ها برای هر کاربری و بافر
        mean_ratio = np.mean(ratios)
        
        # ذخیره میانگین نسبت در دیکشنری
        mean_ratios[landuse_type][buffer_size] = mean_ratio

# چاپ میانگین نسبت‌ها بر اساس کاربری و بافر
for landuse_type in mean_ratios.keys():
    for buffer_size, mean_ratio in mean_ratios[landuse_type].items():
        print(f"میانگین نسبت محاسبه شده برای کاربری {landuse_type} و بافر {buffer_size}: {mean_ratio}")





میانگین نسبت محاسبه شده برای کاربری maskooni و بافر 100: 0.02054414609453699
میانگین نسبت محاسبه شده برای کاربری maskooni و بافر 200: 0.08310825510050744
میانگین نسبت محاسبه شده برای کاربری maskooni و بافر 300: 0.1608907276031779
میانگین نسبت محاسبه شده برای کاربری tejari و بافر 100: 0.08814098137685958
میانگین نسبت محاسبه شده برای کاربری tejari و بافر 200: 0.2221946199393265
میانگین نسبت محاسبه شده برای کاربری tejari و بافر 300: 0.31612880032312807
میانگین نسبت محاسبه شده برای کاربری bayer و بافر 100: 0.047163850400183484
میانگین نسبت محاسبه شده برای کاربری bayer و بافر 200: 0.7047517088676124
میانگین نسبت محاسبه شده برای کاربری bayer و بافر 300: 1.0299397091719198
میانگین نسبت محاسبه شده برای کاربری khadamati و بافر 100: 1.4318115461537766
میانگین نسبت محاسبه شده برای کاربری khadamati و بافر 200: 1.194040222120178
میانگین نسبت محاسبه شده برای کاربری khadamati و بافر 300: 1.0122684869191088
میانگین نسبت محاسبه شده برای کاربری amozshi و بافر 100: 1.0208375887419334
میانگین نسبت محاسبه 